# LangChain Temelleri (Chain Oluşturma)

## 🎯 Bu Kodun Amacı
LangChain framework'ü ile modüler ve yeniden kullanılabilir AI pipeline'ları oluşturmak.

## Ne Yapacağız?
- **ChatGroq** ile LLM bağlantısı kuracağız
- **PromptTemplate** ile parametrik prompt şablonları oluşturacağız
- **StrOutputParser** ile çıktıyı parse edeceğiz
- **LCEL (|)** operatörü ile component'ları **chain** haline getireceğiz
- Aynı şablonu farklı adaylar için tekrar kullanacağız (özgeçmiş özeti örneği)

## Çalıştırma
Kod çalıştırıldığında iki farklı aday için profesyonel özgeçmiş özetleri üretilir.

In [3]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

True

## 🔧 LLM, Template ve Parser Tanımlama

**Ne yapıyoruz?**
- LLM objesi oluşturuyoruz (Groq API ile konuşmak için)
- Parametrik prompt şablonu hazırlıyoruz
- Output parser tanımlıyoruz (string'e çevirmek için)

**llm = ChatGroq(...)**
- LangChain'in Groq wrapper'ı
- temperature=0.7: Orta seviye yaratıcılık

**cv_template = PromptTemplate(...)**
- input_variables: Şablonda kullanılacak değişkenler
- {name}, {position}, {experience}, {skills} → Dinamik olarak doldurulur
- Aynı şablonu farklı verilerle tekrar tekrar kullanabiliriz

**output_parser = StrOutputParser()**
- LLM yanıtını string'e çevirir
- Standardizasyon sağlar

In [4]:

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0.7,
    groq_api_key=os.getenv("GROQ_API_KEY")
)

cv_template = PromptTemplate(
    input_variables=["name", "position", "experience", "skills"],
    template="""Sen bir HR uzmanısın. Aşağıdaki bilgilere göre profesyonel bir özgeçmiş özeti yaz:

Ad: {name}
Pozisyon: {position}
Deneyim: {experience} yıl
Yetenekler: {skills}

3-4 cümlelik, etkileyici bir özet yaz. Pozisyona özel vurgu yap."""
)

output_parser = StrOutputParser()


## 🔗 Chain Oluşturma ve Çalıştırma

**Ne yapıyoruz?**
- Bileşenleri birleştirip bir chain (zincir) oluşturuyoruz
- Test verisi hazırlıyoruz
- Her aday için chain'i çalıştırıp sonuç alıyoruz

**cv_chain = cv_template | llm | output_parser**
- `|` operatörü: Bileşenleri birbirine bağlar (pipe)
- Akış: Template → LLM → Parser
- LCEL (LangChain Expression Language) kullanıyoruz

**candidates listesi**
- İki aday için test verisi
- Her dict, input_variables ile eşleşmeli
- Aynı formatta veri olmalı: name, position, experience, skills

**cv_chain.invoke(candidate)**
- Chain'i tetikler ve tüm akışı çalıştırır
- Candidate dict → template doldurulur → LLM'e gider → string döner
- Aynı şablon, farklı veriler için tekrar kullanılıyor

**Neden chain kullanıyoruz?**
- Tek satırda tüm işlem yapılıyor
- Modüler ve yeniden kullanılabilir
- Kod tekrarı yok

In [5]:
# Chain oluştur
cv_chain = cv_template | llm | output_parser

candidates = [
    {
        "name": "Ayşe Yılmaz",
        "position": "Senior Python Developer",
        "experience": 5,
        "skills": "Python, Django, FastAPI, PostgreSQL, Docker"
    },
    {
        "name": "Mehmet Kaya",
        "position": "Data Scientist",
        "experience": 3,
        "skills": "Python, Machine Learning, TensorFlow, SQL, Statistics"
    }
]

print("ÖZGEÇMIŞ ÖZETLERİ\n")

for candidate in candidates:
    print(f"{candidate['name']} - {candidate['position']}")
    print("-" * 60)
    result = cv_chain.invoke(candidate)
    print(result)
    print()

ÖZGEÇMIŞ ÖZETLERİ

Ayşe Yılmaz - Senior Python Developer
------------------------------------------------------------
Dinamik ve tecrübeli bir Senior Python Developer olarak, 5 yıllık deneyimimle birlikte Python ekosisteminde uzmanlaşmış bir uzmandır. Django ve FastAPI gibi popüler frameworkleri kullanarak ölçeklenebilir ve yüksek performanslı uygulamalar geliştirmekte uzmanım. PostgreSQL ve Docker gibi teknolojileri kullanarak veritabanı yönetimini ve konteynırlaşmayı erfolgreich bir şekilde yönetebilirim. Yüksek kaliteli ve sürdürülebilir yazılımlar üretme konusunda tutkulu bir geliştirici olarak, ekibinize değer katmak için sẵnım.

Mehmet Kaya - Data Scientist
------------------------------------------------------------
Mehmet Kaya, 3 yıllık deneyimine sahip bir Data Scientist olarak, veri analizi ve makine öğrenimi alanlarında uzmanlaşmıştır. Python, Machine Learning, TensorFlow ve SQL gibi güçlü araçlar kullanarak, komplex veri setlerini analiz etme ve içgörüler elde etme konusund